In [5]:
!pip install xmltodict

  Obtaining dependency information for xmltodict from https://files.pythonhosted.org/packages/94/db/fd0326e331726f07ff7f40675cd86aa804bfd2e5016c727fa761c934990e/xmltodict-0.13.0-py2.py3-none-any.whl.metadata


In [45]:
import requests
import xmltodict
import json
import pytz
from datetime import datetime

In [9]:
route_id='455'
onward_calls = 'false'
token = 'UYIFJBFBIV6'

In [12]:
url = f"http://api.rideuta.com/SIRI/SIRI.svc/VehicleMonitor/ByRoute?route={route_id}&onwardcalls={onward_calls}&usertoken={token}"
response = requests.get(url)
xml_data = xmltodict.parse(response.text)

In [37]:
json_data = json.dumps(xml_data)

In [54]:
def convert_to_epoch(timestamp):
    date_part, rest = timestamp.split('.')
    microseconds_part = rest[:-6]  # Extract microseconds part
    timezone_part = rest[-6:]      # Extract timezone part
    microseconds_part = microseconds_part[:6]  # Take only the first 6 digits for microseconds
    timestamp = date_part + '.' + microseconds_part + timezone_part
    dt = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%f%z")
    epoch_time = int(dt.timestamp())
    
    return str(epoch_time)

In [55]:
file_name = route_id + convert_to_epoch(xml_data["Siri"]["ResponseTimestamp"])  + ".json"


In [56]:
import json
with open(file_name, 'w', encoding='utf-8') as f:
   f.write(json_data)

In [26]:
# Opening JSON file
f = open('json_data.json')
data = json.load(f)

In [31]:
xml_data["Siri"]["ResponseTimestamp"]

'2024-08-06T08:30:31.5459909-06:00'